In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

# Create dummy classification data
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_classes=3, random_state=42)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# One-hot encode the target variable
encoder = OneHotEncoder(sparse=False)
y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_encoded = encoder.transform(y_test.reshape(-1, 1))

# Define the FeedForward Neural Network class
class FeedForwardNetwork:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.weights1 = np.random.randn(input_dim, hidden_dim)
        self.bias1 = np.zeros(hidden_dim)
        self.weights2 = np.random.randn(hidden_dim, output_dim)
        self.bias2 = np.zeros(output_dim)

    def forward(self, X):
        self.hidden = np.dot(X, self.weights1) + self.bias1
        self.hidden_activation = self.sigmoid(self.hidden)
        self.output = np.dot(self.hidden_activation, self.weights2) + self.bias2
        self.output_activation = self.softmax(self.output)
        return self.output_activation

    def backward(self, X, y, output_activation, learning_rate):
        batch_size = X.shape[0]
        grad_output = output_activation - y
        grad_weights2 = np.dot(self.hidden_activation.T, grad_output) / batch_size
        grad_bias2 = np.sum(grad_output, axis=0) / batch_size
        grad_hidden_activation = np.dot(grad_output, self.weights2.T)
        grad_hidden = grad_hidden_activation * self.sigmoid_derivative(self.hidden)
        grad_weights1 = np.dot(X.T, grad_hidden) / batch_size
        grad_bias1 = np.sum(grad_hidden, axis=0) / batch_size
        self.weights2 -= learning_rate * grad_weights2
        self.bias2 -= learning_rate * grad_bias2
        self.weights1 -= learning_rate * grad_weights1
        self.bias1 -= learning_rate * grad_bias1

    def train(self, X, y, num_epochs, learning_rate, optimizer):
        costs = []
        for epoch in range(num_epochs):
            output_activation = self.forward(X)
            cost = self.cross_entropy_loss(y, output_activation)
            costs.append(cost)
            self.backward(X, y, output_activation, learning_rate, optimizer)
        return costs

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))

    def softmax(self, x):
        exps = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exps / np.sum(exps, axis=1, keepdims=True)

    def cross_entropy_loss(self, y_true, y_pred):
        epsilon = 1e-8
        y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
        return -np.mean(y_true * np.log(y_pred))

# Define the optimization algorithms
optimizers = {
    'gd': 'Gradient Descent',
    'momentum': 'Momentum',
    'nag': 'Nesterov Accelerated GD',
    'adagrad': 'AdaGrad',
    'rmsprop': 'RMSProp',
    'adam': 'Adam'
}

# Define the hyperparameters
input_dim = X_train.shape[1]
hidden_dim = 100
output_dim = len(np.unique(y_train))
num_epochs = 100
learning_rate = 0.01

# Train the model using different optimizers
for optimizer_name, optimizer_label in optimizers.items():
    # Create the model
    model = FeedForwardNetwork(input_dim, hidden_dim, output_dim)
    # Train the model
    costs = model.train(X_train, y_train_encoded, num_epochs, learning_rate, optimizer_name)
    # Evaluate the model
    y_train_pred = np.argmax(model.forward(X_train), axis=1)
    y_test_pred = np.argmax(model.forward(X_test), axis=1)
    train_accuracy = np.mean(y_train == y_train_pred)
    test_accuracy = np.mean(y_test == y_test_pred)
    # Plot the cost curve
    plt.plot(range(1, num_epochs + 1), costs, label=optimizer_label)
    plt.xlabel('Epochs')
    plt.ylabel('Cost')
    plt.title('Cost Curve')
    plt.legend()

    print(f"Optimizer: {optimizer_label}")
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    print("--------------------------------------")

# Show the plot
plt.show()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


TypeError: ignored